### Feature: Como a planilha orçamentária foi modificada ao longo do tempo

### Features Extracted:

Issue: [#65](https://github.com/lappis-unb/salic-ml/issues/65)
- https://github.com/lappis-unb/salic-ml/issues/65

#### Recarregar automaticamente os módulos

In [1]:
%load_ext autoreload
%autoreload 2

### Importing data

In [2]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats


from salicml.utils.dates import Dates
from core.utils.read_csv import read_csv_with_different_type
from salicml.utils.utils import debug

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

#### Planilha Aprovacão Pai

SQL: [planilha_aprovacao_pai.sql](https://github.com/lappis-unb/salic-ml/blob/master/data/scripts/planilha_aprovacao_pai.sql)


In [3]:
dt_aprovacao_pai = 'planilha_aprovacao_pai.csv'
usecols = ['PRONAC', 'idPlanilhaAprovacao', 'idPlanilhaAprovacaoPai',
       'Item', 'dtPlanilha', 'stAtivo', 'Segmento',]

dtype = {
    'PRONAC': str,
}
dt_aprovacao = read_csv_with_different_type(dt_aprovacao_pai, dtype, usecols=usecols)
dt_aprovacao['idPlanilhaAprovacaoPai'] = dt_aprovacao['idPlanilhaAprovacaoPai'].fillna(value=-1)
dt_aprovacao['idPlanilhaAprovacaoPai'] = dt_aprovacao['idPlanilhaAprovacaoPai'].round(0).astype(int)

display(dt_aprovacao.columns)
dt_aprovacao.head()

Index(['PRONAC', 'idPlanilhaAprovacao', 'idPlanilhaAprovacaoPai', 'Item',
       'dtPlanilha', 'stAtivo', 'Segmento'],
      dtype='object')

,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
0,095259,109198,-1,Contador,2011-03-14 15:31:56.000,S,77
1,093803,792707,-1,Concreto armado para reforços,2013-02-06 11:01:45.687,S,5E
2,093803,1692078,792707,Concreto armado para reforços,2015-11-18 15:11:35.507,N,5E
3,093803,1692134,792763,Madeira: tesoura,2015-11-18 15:11:35.687,N,5E
4,093803,792763,-1,Madeira: tesoura,2013-02-06 11:01:45.687,S,5E


**Convertendo coluna de data de string para datetime**

In [4]:
from datetime import datetime

date_column = 'dtPlanilha'
dt_aprovacao[date_column] = pd.to_datetime(dt_aprovacao[date_column], format = Dates.DATE_INPUT_FORMAT)
dt_aprovacao[date_column] = dt_aprovacao[date_column].values.astype('datetime64[s]')
display(dt_aprovacao.dtypes)

PRONAC                            object
idPlanilhaAprovacao                int64
idPlanilhaAprovacaoPai             int64
Item                              object
dtPlanilha                datetime64[ns]
stAtivo                           object
Segmento                          object
dtype: object

### O caso de idPlanilhaAprovacaoPai = null

No banco de dados há diversos casos `idPlanilhaAprovacaoPai = null`. Há duas observações importantes sobre este caso:

- `idPlanilhaAprovacaoPai = null` identifica itens sem "pai", ou seja, são os itens que ainda não foram modificados, estão na sua primeira versão aprovada.

- O numpy interpreta valores numéricos `null` como `NaN`, o que são convertidos automaticamente para `float`. No banco de dados esta coluna é armazenada como `int`, e tratá-la como `float` é perigoso. Entretanto aparentemente o `numpy` não aceita valores `NaN` como `Int64`. Enfim a solução atual foi converter todos os valoes `NaN` para `-1`, e depois arredondar todos os valores float para `Int64`.

# Dividindo pronacs de treino e pronacs de teste

50/50

In [5]:
from sklearn.model_selection import train_test_split


all_pronacs = dt_aprovacao.PRONAC.unique()
pronacs_train, pronacs_test = train_test_split(all_pronacs, test_size=0.5)


dt_train = dt_aprovacao[dt_aprovacao['PRONAC'].isin(pronacs_train)].copy()
dt_test = dt_aprovacao[dt_aprovacao['PRONAC'].isin(pronacs_test)].copy()

In [6]:
print('pronacs_train = {}'.format(pronacs_train))
print('pronacs_test = {}'.format(pronacs_test))

assert (dt_train.shape[0] + dt_test.shape[0]) == dt_aprovacao.shape[0]

display(dt_train.head())
display(dt_test.head())

pronacs_train = ['130045' '1310144' '137807' ... '114989' '1412291' '164362']
pronacs_test = ['130728' '182349' '160681' ... '123924' '145683' '150339']


,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
1,093803,792707,-1,Concreto armado para reforços,2013-02-06 11:01:45,S,5E
2,093803,1692078,792707,Concreto armado para reforços,2015-11-18 15:11:35,N,5E
3,093803,1692134,792763,Madeira: tesoura,2015-11-18 15:11:35,N,5E
4,093803,792763,-1,Madeira: tesoura,2013-02-06 11:01:45,S,5E
5,110895,124708,-1,Maestro,2011-04-05 15:11:20,S,85


,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
0,095259,109198,-1,Contador,2011-03-14 15:31:56,S,77
8,095195,820076,-1,Ilustração,2013-03-06 09:26:02,S,53
11,093242,37564,-1,Material de escritório,2011-02-07 17:04:15,S,51
23,091695,116491,-1,Assistente de produção,2011-03-21 11:17:31,S,71
26,101800,16956,-1,Material de consumo,2011-02-07 16:37:02,S,52


In [7]:
dt_train.sort_values(by=['PRONAC'], inplace=True)
dt_train.head(5)

,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
553,090226,136170,-1,Hospedagem com alimentação,2011-04-07 18:13:42,S,71
1273560,090226,136152,-1,Sítio de Internet - Registro de domínio,2011-04-07 18:13:42,S,71
2108803,090226,136161,-1,Coordenador do projeto,2011-04-07 18:13:42,S,71
1188310,090226,136146,-1,Revisão de texto,2011-04-07 18:13:42,S,71
1021505,090226,136149,-1,Coordenação editorial,2011-04-07 18:13:42,S,71


In [8]:
dt_pronac = dt_train.groupby(by=['PRONAC'])

In [9]:
PRONAC = 'PRONAC'
ITEM_ID = 'idPlanilhaAprovacao'
PARENT_ID = 'idPlanilhaAprovacaoPai'
ST_ATIVO = 'stAtivo'
NULL_PARENT = -1

In [10]:
def print_tree(parent, node):
    """ Prints path from leaf to root (in a list form)"""
    print(node, end='')
    father = parent[node]
    if father != NULL_PARENT:
        print(' -> ', end='')
        print_tree(parent, father)
    else:
        print()  
        

def print_forest(pronac, parent, leaves):
    print('PRONAC = [{}]\n'.format(pronac))
    print('parent = {}\n'.format(parent))
    print('leaves = {}\n'.format(leaves))
    print('\n')
    for leaf in leaves:
        print_tree(parent, leaf)

In [20]:
def init_parent_data(rows):
    parent = {}
    leaves = set(rows.idPlanilhaAprovacao.unique())

    for index, row in rows.iterrows():
        pronac = row[PRONAC]
        item_id = row[ITEM_ID]
        parent_id = row[PARENT_ID]
        parent[item_id] = parent_id
        
        try:
            leaves.remove(parent_id)
        except KeyError:
            """parent_id was removed before or its a NULL_PARENT"""
            pass
    return parent, leaves

def get_number_of_spreadsheet_modifications(items, leaves):
    all_items_ids = rows.idPlanilhaAprovacao.unique()
    modifications = all_items_ids.size - len(leaves)
    return modifications

def get_number_of_spreadsheets(items):
    date_group = items.groupby(['PRONAC', 'dtPlanilha']).count()
    display(date_group)
    return len(date_group)
    

pronac = '1310741'
rows = dt_train[dt_train.PRONAC == pronac]
#display(rows)

parent, leaves = init_parent_data(rows)
print_forest(pronac, parent, leaves)

modifications = get_number_of_spreadsheet_modifications(rows, leaves)
print('number of spreadsheet modifications = {}'.format(modifications))

number_spreadsheets = get_number_of_spreadsheets(rows)
print('number of spreadsheets = {}'.format(number_spreadsheets))

PRONAC = [1310741]

parent = {1107669: -1, 1108239: -1, 2136593: 1107782, 2136978: 1108167, 1108287: -1, 2136594: 1107783, 2136484: 1107673, 2136598: 1107787, 1107774: -1, 2136585: 1107774, 1107787: -1, 2136974: 1108163, 2136760: 1107949, 1107782: -1, 2136661: 1107850, 2136832: 1108021, 1107673: -1, 1107715: -1, 1107912: -1, 2136616: 1107805, 2137040: 1108229, 2136889: 1108078, 1108254: -1, 1108282: -1, 1108078: -1, 2136970: 1108159, 1107850: -1, 2137098: 1108287, 2137093: 1108282, 2136500: 1107689, 2137050: 1108239, 1108013: -1, 1107783: -1, 1108021: -1, 2136885: 1108074, 2136480: 1107669, 1107674: -1, 1108159: -1, 1107949: -1, 2136378: 1107567, 2136824: 1108013, 2136299: 1107488, 2136816: 1108005, 2136447: 1107636, 1107636: -1, 2136433: 1107622, 2136395: 1107584, 1107805: -1, 1107584: -1, 1108088: -1, 2136933: 1108122, 2136899: 1108088, 2136590: 1107779, 1107515: -1, 2136808: 1107997, 1107543: -1, 1107997: -1, 2137100: 1108289, 1107779: -1, 2136842: 1108031, 1108031: -1, 1107622: -1,

2137010 -> 1108199
2137011 -> 1108200
2137012 -> 1108201
2137013 -> 1108202
2137014 -> 1108203
2137015 -> 1108204
2137016 -> 1108205
2137017 -> 1108206
2137018 -> 1108207
2137019 -> 1108208
2137020 -> 1108209
2137021 -> 1108210
2137022 -> 1108211
2137023 -> 1108212
2137024 -> 1108213
2137025 -> 1108214
2137026 -> 1108215
2137027 -> 1108216
2137028 -> 1108217
2137029 -> 1108218
2137030 -> 1108219
2137031 -> 1108220
2137032 -> 1108221
2137033 -> 1108222
2137034 -> 1108223
2137035 -> 1108224
2137036 -> 1108225
2137037 -> 1108226
2137038 -> 1108227
2137039 -> 1108228
2137040 -> 1108229
2137041 -> 1108230
2137042 -> 1108231
2137043 -> 1108232
2137044 -> 1108233
2137045 -> 1108234
2137046 -> 1108235
2137047 -> 1108236
2137048 -> 1108237
2137049 -> 1108238
2137050 -> 1108239
2137051 -> 1108240
2137052 -> 1108241
2137053 -> 1108242
2137054 -> 1108243
2137055 -> 1108244
2137056 -> 1108245
2137057 -> 1108246
2137058 -> 1108247
2137059 -> 1108248
2137060 -> 1108249
2137061 -> 1108250
2137062 -> 1

idPlanilhaAprovacao  idPlanilhaAprovacaoPai  \
PRONAC  dtPlanilha                                                         
1310741 2013-12-05 12:05:20                    1                       1   
        2013-12-05 12:05:21                  362                     362   
        2013-12-05 12:05:22                  376                     376   
        2013-12-05 12:05:23                   85                      85   
        2017-03-21 15:38:47                  126                     126   
        2017-03-21 15:38:48                  293                     293   
        2017-03-21 15:38:49                  296                     296   
        2017-03-21 15:38:50                  109                     109   

                             Item  stAtivo  Segmento  
PRONAC  dtPlanilha                                    
1310741 2013-12-05 12:05:20     1        1         1  
        2013-12-05 12:05:21   362      362       362  
        2013-12-05 12:05:22   376      376       376  
        2013-12-05 12:05:23    85       85        85  
        2017-03-21 15:38:47   126      126       126  
        2017-03-21 15:38:48   293      293       293  
        2017-03-21 15:38:49   296      296       296  
        2017-03-21 15:38:50   109      109       109

number of spreadsheets = 8


In [18]:
dt_train.groupby(['PRONAC', 'stAtivo']).count().sort_values(by='idPlanilhaAprovacao', ascending=False).head(20)

,,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,Segmento
PRONAC,stAtivo,,,,,
112006,S,3620,3620,3620,3620,3620
1113323,S,2180,2180,2180,2180,2180
172085,N,1476,1476,1476,1476,1476
111293,S,1447,1447,1447,1447,1447
155152,N,1326,1326,1326,1326,1326
1411314,N,1307,1307,1307,1307,1307
128136,S,1245,1245,1245,1245,1245
1410754,S,1241,1241,1241,1241,1241
137225,S,963,963,963,963,963


In [13]:
segment_projects = dt_train[['PRONAC', 'idSegmento', 'idComprovantePagamento']].groupby(['idSegmento', 'PRONAC']).nunique()
segment_projects.drop(columns=['PRONAC', 'idSegmento'], inplace=True)
segment_projects.rename(columns={'idComprovantePagamento': 'NumeroComprovantes'}, inplace=True)

display(segment_projects.head())

KeyError: "['idSegmento' 'idComprovantePagamento'] not in index"

### Plotando as distribuiçoes dos segmentos mais comuns

In [ ]:
common_segements = segment_projects.groupby(['idSegmento']).count()
common_segements.rename(columns={'NumeroComprovantes': 'NumeroProjetos'}, inplace=True)
common_segements.sort_values(by=['NumeroProjetos'], ascending=False, inplace=True)
display(common_segements.head())

In [ ]:
from scipy.stats import norm


def plot_segment(id, receipts):
    plt.hist(receipts, bins=100, density=True, stacked=True, alpha=0.6, color='g', edgecolor='black')
    #plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.title('Segmento "{}"'.format(id))
    plt.xlabel('Número de recibos por projeto')
    plt.ylabel('Frequência de projetos')

    mu, std = np.mean(receipts), np.std(receipts)
    
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 200)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)
    plt.show()

for counter, id_segmento in enumerate(common_segements.index.values):    
    projects_total_receipts = segment_projects.loc[id_segmento].NumeroComprovantes.values
    plot_segment(id_segmento, projects_total_receipts)
    
    counter += 1
    if counter == 20:
        break

In [ ]:
arr = segment_projects['NumeroComprovantes'].values
plot_segment('all', arr)

In [ ]:
segment_receipts_avg_std = segment_projects.groupby(['idSegmento'])
segment_receipts_avg_std = segment_receipts_avg_std.agg(['count', 'sum', 'mean', 'std'])

segment_receipts_avg_std.columns = segment_receipts_avg_std.columns.droplevel(0)

display(segment_receipts_avg_std.head())

# Calculando a porcentagem de outliers no conjunto de teste

In [ ]:
project_receipts_grp = dt_items[['PRONAC', 'idComprovantePagamento', 'idSegmento']].groupby(['PRONAC'])
project_receipts = project_receipts_grp.nunique()
project_receipts.drop(columns=['PRONAC', 'idSegmento'], inplace=True)
project_receipts.rename(columns={'idComprovantePagamento': 'NumeroComprovantes'}, inplace=True)
display(project_receipts.head())

In [ ]:
id_segmento = '11'
display(segment_receipts_avg_std.loc[id_segmento])
mean = segment_receipts_avg_std.loc[id_segmento]['mean']
print('mean = {}'.format(mean))

In [ ]:
from salicml.outliers.gaussian_outlier import is_outlier


def is_total_receipts_outlier(pronac):
    assert isinstance(pronac, int)
    
    total_receipts = project_receipts.loc[pronac]['NumeroComprovantes']
    id_segmento = project_receipts_grp.get_group(pronac).iloc[0]['idSegmento']
    
    if not np.isin(id_segmento, segment_receipts_avg_std.index):
        raise ValueError('Segment {} was not trained'.format(id_segmento))
    
    mean = segment_receipts_avg_std.loc[id_segmento]['mean']
    std = segment_receipts_avg_std.loc[id_segmento]['std']
    outlier = is_outlier(total_receipts, mean, std)
    return outlier

pronac = int(np.random.choice(dt_test.PRONAC.values))
print('pronac = {}'.format(pronac))
is_total_receipts_outlier(pronac)

In [ ]:
from salicml.outliers.gaussian_outlier import outlier_probability


print(outlier_probability(5011203123, 113132, c=1.5))
print(outlier_probability(5, 132, c=1.5))
print(outlier_probability(-1, 3, c=1.5))

In [ ]:
pronacs_test = dt_test.PRONAC.unique()

outlier_arr = []

for i, pronac in enumerate(pronacs_test):
    try:
        outlier = is_total_receipts_outlier(int(pronac))
        outlier_arr.append(1.0 if outlier else 0.0)
    except ValueError as err:
        print(err)
    
describe = pd.DataFrame(outlier_arr).describe()
display(describe)

# Análise dos resultados

Para `c = 1.5`, esperava-se que `6.68%` dos projetos fossem considerados outliers em termos dos números de itens por projeto. Entretanto, para os conjuntos de treino e teste utilizados e `c = 1.5`, `7.57%`dos projetos foram considerados outliers.

# Contribuições futuras

A celula anterior demonstra um problema na divisão dos pronacs nos conjuntos de treino e teste: alguns segmentos, por exemplo o segmento `9I` ficaram fora do conjunto do treino, o que não permite detectar anomalias para este segmento e os demais que não foram incluidos no conjunto de treino. Portanto uma contribução futura desejável é que o conjunto de treino garanta que para segmento, pelo menos 50% do total dos projetos para aquele segmento esteja dentro do conjunto de pronacs de treino.